# Encoding Python Objects with JSON

This guide will walk through creating custom encoders when dumping dictionaries to JSON objects. 

Let us create a dictionary containing nested python objects.  The JSON module can only convert basic python types by default.  In this notebook, I plan to demonstrate how to create a custom encoder to dump python dictionaries that contain object types. 

Below is an table of the default supported types:

| Python      | JSON        |
| ----------- | ----------- |
| dict      | ojbect        |
| list, tuple   | array        |
|str  | str        |
|int, float, int- & float-derived Enums  | number    |
|True  | true        |
|False  | false        |
|None  | null        |




Custom Python objects will have an issue serializing and will require custom encoders to process those types. 

First, let's set up the environment by importing the JSON module. In this example, our dictionary will contain an item with a custom class called "Decimal."  When invoking the JSON dumps method, we will illustrate what happens when you do not have a custom decoder configured.  



In [38]:
import json 

# Step 1
class Decimal:
    def __init__(self, num):
        self.num = num
        
    def __repr__(self):
        return f'< Decimal(num={self.num}) >'

    
# Step 2
my_decimal = Decimal(5)

data = {
    "item1": my_decimal,
    "item2": 5
}

try:
    json.dumps(data)
except TypeError as err:
  
    
    print(str(err))
    


Object of type Decimal is not JSON serializable


The message above describes the TypeError exception raising. The is due to the "item1 key" containing the value of an instance of a Decimal object. 

Next, let us create a serialization function that will be able to handle the encoding to JSON. This function will check if the object is of the correct type. The function will return the converted basic python type. 

Let's pass the newly created serialize function to the 'default' parameter to the JSON dumps function call.

In [43]:
def encode(obj):
    if isinstance(obj, Decimal):
        return int(obj.num)
    return obj.__dict__

    
json_str = json.dumps(data, default=encode)
print(json_str)
type(json_str)

{"item1": 5, "item2": 5}


str

The above output displays the python type of the resulting JSON dumps calls as a string instead of a dictionary.  Binary data is transferred across the wire when using JSON. JSON represents its data as a human-readable string.  If you would like to access the properties of the JSON object, you will not be able to do so unless you reconvert the string back into a python object.